This notebook is written to implement the model for Epistasis as defined in the research work titled 'Computational Framework for Statistical Epistasis Supports XOR Penetrance Function in a Living System'. 

In [40]:
# importing all the required packages
import pandas as pd
import numpy as np
import scipy.stats
import math
from time import process_time_ns
import statsmodels.stats.multitest as ssm

In [43]:
# reading the data - A sample dataset having 100 SNPs and phenotype BMI for 5566 samples is being used. This is just a sample and maynot be having significant epistatic pairs.
data = pd.read_csv("sample_data.csv") # the last column contains the phenotype values, in this case bmi
phenotype = data.iloc[:,-1] # extracting the phenotype

In [44]:
# defining and initializing required variables

# defining the significance level 
alpha = .05

# storing the count of number of samples in the variable n
n = data.shape[0]

# defining degrees of freedom where n is number of samples
df = n-4

# defining the critical value
critical_value = scipy.stats.t.ppf(q=1-alpha/2,df=df)


Below is the algoritm implementation for the partial correlation of interaction of genes for Epistasis. We use two different ways of encoding the interaction terms. Firstly, the traditional method of cartesian product and then by using the XOR penetrance function.

Algorithm 1- Algorithm for 2 way epistasis detection

In [52]:
# Algorithm for Interaction Coefficient for Pairwise Epistasis
"""Parameters for the method:
    snp1 - First snp in the pair to be checked for interaction
    snp2 - Second snp in the pair to be checked for interaction
    phenotype - Phenotype vector
"""
def epistatis(snp1, snp2, phenotype): 
    try:
        # removing the intercept by mean centering
        snp1_tilde = snp1 - snp1.mean()
        snp2_tilde = snp2 - snp2.mean()
        phenotype_tilde = phenotype - phenotype.mean()

        # declaring the interaction vector - will contain the interaction term of snp1 and snp2
        interaction_vector = pd.Series(dtype='float64')
    
        # defining the interaction vector - either using cartesian product or XOR
        interaction_vector = (snp1%2 + snp2%2)%2 # using XOR penetrance
        #interaction_vector = snp1.mul(snp2) # using cartesian product
    
        # mean centering the interaction vector 
        interaction_vector_tilde = interaction_vector - interaction_vector.mean()
   
        # computing the dot products as explained in the algorithm
        x = (snp1_tilde.dot(interaction_vector_tilde)/snp1_tilde.dot(snp1_tilde)) * (snp1_tilde) # 2nd term in the v variable, using to breakdown the formula
        v = interaction_vector_tilde - x
        q2 = snp2_tilde - (((snp1_tilde.dot(snp2_tilde)) / (snp1_tilde.dot(snp1_tilde))) * snp1_tilde)
        v = v - ((interaction_vector_tilde.dot(q2)/q2.dot(q2))*q2)
        b3 = (v.dot(phenotype_tilde)) / (v.dot(v)) # interactionn coefficient, referred as beta_3 in the paper
     
        t_test = np.sqrt(df)*b3/(np.sqrt(1-b3*b3))
        p_val = scipy.stats.t.sf(abs(t_test), df)
    except Exception as e:
        print("Error pair detected with error: ", e)
        b3 = 0
        t_test = 0
        p_val = 1

    return b3, t_test, p_val # returning the interaction coefficient,, t test value and p value

In [ ]:
# applying the above defined method to the dataset

p_value_locus = [] # list of tuples containing the p_value and the two interacting loci
for i in range(0, data.shape[1]-1):
    for j in range(i+1, data.shape[1]-1):
        interacting_snp_1 = data.iloc[:,i]
        interacting_snp_2 = data.iloc[:,j]
        b3, t_test, p_val = epistatis(interacting_snp_1, interacting_snp_2, phenotype)
        p_value_locus.append((p_val, data.columns[i], data.columns[j]))

        
        # printing the interacting pairs with ttest values greater than the selected critical value
        if abs(t_test) > critical_value:
            print(" Interacting SNP 1 = {0} \t  Interacting SNP 2 ={1} \t  Beta Coefficient(r) = {2}  \t t_test = {3} \t p_val = {4} ".format(data.columns[i], data.columns[j], b3, t_test, p_val))

        # printing exception cases
        if b3==0 and t_test==0 and p_val==1:
            print("Error Pair: ")
            print(" Interacting SNP 1 = {0} \t  Interacting SNP 2 ={1} \t  Beta Coefficient(r) = {2}  \t t_test = {3} \t p_val = {4} ".format(data.columns[i], data.columns[j], b3, t_test, p_val))

In [54]:
# performing fdr correction to correct for multiple tests

all_p_values = [] # list of all the p_values of the pairwise combinations
for i in range(0, len(p_value_locus)):
    all_p_values.append(p_value_locus[i][0])

accept, corrected_pvals = ssm.fdrcorrection(np.asarray(all_p_values).flatten(), alpha=0.001)

for i in range(0, len(corrected_pvals)):
    # printing the significant pairs after p value correction
    if corrected_pvals[i] < 0.001:
        print(" p-value = {0} \t adjusted p-value = {1} \t loci 1 = {2} \t loci 2 = {3} ".format( p_value_locus[i][0],  corrected_pvals[i], p_value_locus[i][1], p_value_locus[i][2]))

Below is the algorithm implementation for 3 way epistasis detection, it can be generalized to k-wise epistasis.
Algorithm 2- Algorithm implementation for 3 way epistasis detection

In [55]:
def epistatis_three_way(i, j, k, phenotype):
    try:
        snp = np.matrix(data.iloc[:,:-1]) # reading the SNP data
        phenotype_tilde = phenotype - phenotype.mean() # reading the phenotype data
        phenotype_tilde = np.matrix(phenotype_tilde)
        
        x = np.transpose(np.matrix(snp[:,i])) # interacting SNP1
        y = np.transpose(np.matrix(snp[:,j])) # interacting SNP2
        z = np.transpose(np.matrix(snp[:,k])) # interacting SNP3
        
        # Calculation of the variable q(Q)
        q = np.concatenate((x,y,z), axis=1) 
        a = np.multiply(x,y)

        q = np.concatenate((q,a),axis=1)
        a = np.multiply(y,z)

        q = np.concatenate((q,a),axis=1)
        a = np.multiply(x,z)

        q = np.concatenate((q,a),axis=1)
        a = np.multiply(x,y)
        a = np.multiply(a,z)
        q = np.concatenate((q,a),axis=1)
        q = q - q.mean(axis=0)
        q = np.concatenate((q,phenotype_tilde),axis=1)

        # beta for interaction of 3way epistasis after q is calculated
        beta = 0
        for l in range(1,8):
          gs_curr = q[:,l]
          for m in range(l):
            gs_prev = q[:,m]
            curr = np.matmul(np.transpose(gs_curr), gs_prev)/np.matmul(np.transpose(gs_prev), gs_prev)
            if l == 7 and m == 6:
              beta = curr
            q[:,l] = q[:,l] - curr[0,0]*gs_prev
        v = q[:,6]
        resid = q[:,7]
        dot_v_v = np.matmul(np.transpose(v),v)[0,0]
        dot_resid_resid = np.matmul(np.transpose(resid),resid)[0,0]
        t_test = np.sqrt(df)*beta*np.sqrt(dot_v_v)/np.sqrt(dot_resid_resid)
        t_test = t_test[0,0]
        p_val = 2*scipy.stats.t.sf(abs(t_test), df)

    except Exception as e:
        print("Error pair detected with error: ", e)
        t_test = 0
        p_val = 1

    return t_test, p_val


In [ ]:
# applying the above defined method (3 way Epistasis) to the dataset

p_value_locus = [] # list of tuples containing the p_value and the two interacting loci
for i in range(0, data.shape[1]-1):
    for j in range(i+1, data.shape[1]-1):
        for k in range(j+1, data.shape[1]-1):
            
            t_test, p_val = epistatis_three_way(i, j, k, phenotype)
            p_value_locus.append((p_val, data.columns[i], data.columns[j], data.columns[k]))

            # print(" Interacting SNP 1 = {0} \t  Interacting SNP 2 ={1} \t Interacting SNP 3 = {2}  \t t_test = {3} \t p_val = {4} ".format(data.columns[i], data.columns[j], data.columns[k], t_test, p_val))
            # printing the interacting pairs with ttest values greater than the selected critical value
            if abs(t_test) > critical_value:
                print(" Interacting SNP 1 = {0} \t  Interacting SNP 2 ={1} \t Interacting SNP 3 = {2}  \t t_test = {3} \t p_val = {4} ".format(data.columns[i], data.columns[j], data.columns[k], t_test, p_val))

            # printing exception cases
            if t_test==0 and p_val==1:
                print("Error Pair: ")
                print(" Interacting SNP 1 = {0} \t  Interacting SNP 2 ={1} \t Interacting SNP 3 = {2}  \t t_test = {3} \t p_val = {4} ".format(data.columns[i], data.columns[j], data.columns[k], t_test, p_val))

Below is the implementation of the permutation testing algorithm, which helps to validate the detected epistasis pairs.
Algorithm 3- Permutation Testing Algorithm

In [35]:

# defining the significance level 
alpha = .001

# storing the count of number of samples in the variable n
n = data.shape[0]

# number of snps 
m = data.shape[1]

# defining degrees of freedom where n is number of samples
df = n-4

# defining the critical value
critical_value = scipy.stats.t.ppf(q=1-alpha/2,df=df)

In [57]:
# making the permuted phenotype matrix. Each column will have a version of the permutated phenotype. 
# Assuming your phenotype column is named 'y' in your DataFrame 'data

phenotype_column = data['y']
K = 10 # number of times to be permutated

# Create an empty matrix P to store permuted phenotype matrices
P = np.zeros((len(phenotype_column), K))

# Iterate over each permutation
for i in range(K):
    # Shuffle the values in the phenotype column
    permuted_phenotype = np.random.permutation(phenotype_column)
    
    # Store the permuted phenotype in the corresponding column of P
    P[:, i] = permuted_phenotype
P_df = pd.DataFrame(P)

In [ ]:
# changing data to numpy array from dataframe for easier mathematical manipulations
data = data.to_numpy()
# centering the data
data_tilde = data - data.mean()

tilde_exp = data_tilde # defining variable tilde_exp to hold the centered data
cols = data.shape[1] # number of columns in the data, i.e number of SNPs in the data

tilde_p = phenotype - phenotype.mean() # centering the phenotype

permuted_p = P # permutated phenotype matrix
num_tests = K # number of permutation tests

detected = 0 # to ocunt the number of pairs detetcted
results = pd.DataFrame([[0,0,0,0]], columns=['pval','ttest','snp1','snp2'])
for i in range(cols):
    for j in range(i+1,cols):
        try:
           
           # Encoding interaction for epistasis
           # traditional way to just multiple vectors for loci
           #tilde_i = np.multiply(snp[:,i], snp[:,j])
           # using xor penetrance function
           #tilde_i = (snp[:,i]%2+snp[:,j]%2)%2
           tilde_i = (data[:,i]%2+data[:,j]%2)%2
           tilde_i = tilde_i - np.mean(tilde_i, axis=0)
           print(tilde_i)


           # check_epistasis(g_i,g_j, I_ij) - all code below would be this
           dot_gi_ti = np.dot(tilde_i, tilde_exp[:,i])
           dot_gi_p = np.dot(tilde_p, tilde_exp[:,i])
           dot_gi_gi = np.dot(tilde_exp[:,i], tilde_exp[:,i])

           v = tilde_i - dot_gi_ti/dot_gi_gi*tilde_exp[:,i]
           resid = tilde_p - dot_gi_p/dot_gi_gi*tilde_exp[:,i]

           dot_gj_ti = np.dot(tilde_i, tilde_exp[:,j])
           dot_gj_gj = np.dot(tilde_exp[:,j], tilde_exp[:,j])
           dot_gi_gj = np.dot(tilde_exp[:,i], tilde_exp[:,j])
           q_2 = tilde_exp[:,j]-dot_gi_gj/dot_gi_gi*tilde_exp[:,i]

           dot_q2_ti = np.dot(tilde_i, q_2)
           dot_q2_p = np.dot(tilde_p, q_2)
           dot_q2_q2 = np.dot(q_2, q_2)
           # residual of I
           v = v-dot_q2_ti/dot_q2_q2*q_2 # gives the residual I from the paper
           resid = resid - dot_q2_p/dot_q2_q2*q_2 #  residual calculated as per algorithm in the paper

           # dot product of I with it's residual
           dot_v_v = np.dot(v,v) 
           dot_v_p = np.dot(v,tilde_p)

           #direct-test for beta_3 called r below
           r = dot_v_p/dot_v_v
           resid = resid - r*v
           dot_resid_resid = np.dot(resid,resid)
           
           t_test = np.sqrt(df)*r*np.sqrt(dot_v_v)/np.sqrt(dot_resid_resid)


           #permutation testing for pvals now
           # first loci dot products
           perm_gi = np.dot(permuted_p.T,tilde_exp[:,i].reshape(n,1))/dot_gi_gi 
           resid_p = permuted_p - np.outer(tilde_exp[:,i].reshape(n,1),perm_gi) 
           perm_gj = np.dot(permuted_p.T,q_2)/dot_q2_q2
           resid_p = resid_p - np.outer(q_2,perm_gj) 
           perm_i = np.dot(permuted_p.T,v).reshape(num_tests,1) 
           print("perm beta3s")
           print("perm beta3s: \n", perm_i/dot_v_v) 
           resid_p = resid_p - np.outer(tilde_i,perm_i/dot_v_v) 
           test_betas = np.sum(np.multiply(permuted_p, resid_p),axis=0).reshape(num_tests,1) 
           test_residp= np.sum(np.multiply(permuted_p, permuted_p),axis=0).reshape(num_tests,1) 
           test_betas = np.sqrt(dot_v_v*test_betas)
           test_betas = np.sqrt(df)*np.divide(perm_i,test_betas)
           p_val = sum(sum(abs(test_betas) >= abs(t_test)))/num_tests
        except Exception as e:
           print("Error pair (",i, ",",j,"):",e)
           t_test = 0
           p_val = 1
        df2 = pd.DataFrame([[p_val,t_test,i,j]], columns=['pval','ttest','snp1','snp2'])
        results = pd.concat([results,df2]) 
        if abs(t_test) > critical_value and p_val < alpha:
            detected = detected + 1
t2 = process_time_ns()
results.to_csv("perm12_test.csv")
print("number of detected pairs:", detected)
exit()